In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

C:\Users\yfsta\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,...,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,Bookkeeper/Accounting,10+ years,MORTGAGE,48000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,297xx,SC,33.18,0.0,Jun-1991,0.0,33.0,2.0,11.0,2.0,19108.0,29.6%,19.0,w,13668.88,13668.88,4519.68,4519.68,...,0.0,0.0,1.0,6.0,2848.0,263953.0,17.6,0.0,0.0,131.0,294.0,11.0,6.0,2.0,11.0,76.0,11.0,76.0,2.0,6.0,9.0,6.0,8.0,6.0,9.0,11.0,9.0,11.0,0.0,0.0,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,tech,8 years,MORTGAGE,60000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,299xx,SC,22.44,0.0,Jun-1996,0.0,NaN,NaN,7.0,0.0,7722.0,59.4%,9.0,w,6635.69,6635.69,3572.97,3572.97,...,0.0,0.0,2.0,2.0,7912.0,4778.0,61.8,0.0,0.0,234.0,91.0,9.0,9.0,0.0,11.0,NaN,9.0,NaN,0.0,3.0,3.0,3.0,3.0,5.0,4.0,4.0,3.0,7.0,0.0,0.0,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,Sales Manager,10+ years,MORTGAGE,109000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,226xx,VA,26.02,0.0,Dec-2001,1.0,NaN,NaN,9.0,0.0,20862.0,54.3%,19.0,w,0.00,0.00,26224.23,26224.23,...,1.0,0.0,1.0,2.0,33976.0,17538.0,54.3,0.0,0.0,142.0,168.0,13.0,13.0,3.0,13.0,NaN,0.0,NaN,0.0,3.0,3.0,5.0,6.0,7.0,5.0,9.0,3.0,9.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,Senior Manager,10+ years,MORTGAGE,92000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,275xx,NC,21.60,0.0,May-1984,0.0,42.0,NaN,16.0,0.0,51507.0,64.5%,24.0,w,19263.77,19263.77,10271.36,10271.36,...,0.0,1.0,0.0,1.0,13819.0,16623.0,59.9,0.0,0.0,82.0,379.0,19.0,19.0,2.0,19.0,48.0,NaN,42.0,0.0,7.0,9.0,9.0,11.0,4.0,13.0,18.0,9.0,16.0,0.0,0.0,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,Program Coordinator,8 years,RENT,55000.0,Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,462xx,IN,25.49,0.0,Mar-2005,4.0,NaN,30.0,18.0,1.0,9568.0,46%,19.0,w,0.00,0.00,9190.49,9190.49,...,2.0,0.0,5.0,17.0,1051.0,1375.0,45.0,0.0,0.0,129.0,95.0,0.0,0.0,0.0,8.0,NaN,0.0,NaN,0.0,2.0,17.0,2.0,2.0,2.0,17.0,17.0,13.0,18.0,NaN,0.0,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical colummn and see what happens off of this kind of naive approach.

In [4]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
#X = pd.get_dummies(X)

#cross_val_score(rfc, X, Y, cv=5)

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [5]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [6]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [7]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,...,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,RENT,35000.0,Verified,Jan-2015,Charged Off,n,debt_consolidation,Debt consolidation,30.90,0.0,0.0,NaN,84.0,9.0,1.0,11031.0,22.0,w,0.00,0.00,5034.02,5034.02,2527.70,2506.32,0.0,0.00,0.0000,May-2016,316.07,NaN,...,NaN,NaN,NaN,7.0,3798.0,2554.0,75.9,0.0,0.0,134.0,108.0,6.0,6.0,0.0,17.0,NaN,10.0,NaN,0.0,4.0,6.0,4.0,4.0,12.0,7.0,10.0,6.0,9.0,0.0,0.0,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,RENT,64400.0,Source Verified,Jan-2015,Charged Off,n,debt_consolidation,Debt consolidation,27.19,1.0,2.0,22.0,NaN,17.0,0.0,8254.0,20.0,w,0.00,0.00,5366.99,5366.99,1552.69,2234.98,0.0,1579.32,284.2776,Jan-2016,317.86,NaN,...,NaN,NaN,NaN,6.0,3895.0,3752.0,61.3,0.0,0.0,56.0,83.0,5.0,5.0,0.0,12.0,NaN,0.0,NaN,1.0,5.0,7.0,7.0,7.0,3.0,15.0,16.0,7.0,17.0,0.0,0.0,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,RENT,100000.0,Verified,Jan-2015,Current,n,credit_card,Credit card refinancing,10.83,0.0,1.0,NaN,110.0,8.0,1.0,33266.0,10.0,f,7475.67,7475.67,15966.05,15966.05,12524.33,3441.72,0.0,0.00,0.0000,Jan-2017,664.20,Feb-2017,...,NaN,NaN,NaN,1.0,4163.0,8434.0,79.8,0.0,0.0,146.0,155.0,5.0,5.0,0.0,5.0,NaN,0.0,NaN,0.0,5.0,5.0,6.0,7.0,1.0,6.0,7.0,5.0,8.0,0.0,0.0,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [9]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,...,last_pymnt_d_Nov-2016,last_pymnt_d_Oct-2015,last_pymnt_d_Oct-2016,last_pymnt_d_Sep-2015,last_pymnt_d_Sep-2016,next_pymnt_d_Feb-2017,next_pymnt_d_Jan-2017,next_pymnt_d_Jul-2016,next_pymnt_d_Mar-2017,last_credit_pull_d_Apr-2015,last_credit_pull_d_Apr-2016,last_credit_pull_d_Aug-2015,last_credit_pull_d_Aug-2016,last_credit_pull_d_Dec-2014,last_credit_pull_d_Dec-2015,last_credit_pull_d_Dec-2016,last_credit_pull_d_Feb-2015,last_credit_pull_d_Feb-2016,last_credit_pull_d_Jan-2015,last_credit_pull_d_Jan-2016,last_credit_pull_d_Jan-2017,last_credit_pull_d_Jul-2015,last_credit_pull_d_Jul-2016,last_credit_pull_d_Jun-2015,last_credit_pull_d_Jun-2016,last_credit_pull_d_Mar-2015,last_credit_pull_d_Mar-2016,last_credit_pull_d_May-2015,last_credit_pull_d_May-2016,last_credit_pull_d_Nov-2015,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401.0,72868139.0,16000.0,16000.0,16000.0,14.85,379.39,48000.0,33.18,0.0,0.0,33.0,2.0,11.0,2.0,19108.0,19.0,13668.88,13668.88,4519.680000,4519.68,2331.12,2188.56,0.0,0.00,0.0000,379.39,0.0,33.0,1.0,NaN,NaN,0.0,0.0,31329.0,1.0,2.0,1.0,2.0,6.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,68354783.0,73244544.0,9600.0,9600.0,9600.0,7.49,298.58,60000.0,22.44,0.0,0.0,NaN,NaN,7.0,0.0,7722.0,9.0,6635.69,6635.69,3572.970000,3572.97,2964.31,608.66,0.0,0.00,0.0000,298.58,0.0,NaN,1.0,NaN,NaN,0.0,0.0,55387.0,0.0,3.0,0.0,0.0,47.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,68466916.0,73356753.0,25000.0,25000.0,25000.0,7.49,777.55,109000.0,26.02,0.0,1.0,NaN,NaN,9.0,0.0,20862.0,19.0,0.00,0.00,26224.230000,26224.23,25000.00,1224.23,0.0,0.00,0.0000,20807.39,0.0,NaN,1.0,NaN,NaN,0.0,0.0,305781.0,0.0,3.0,0.0,1.0,13.0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,68466961.0,73356799.0,28000.0,28000.0,28000.0,6.49,858.05,92000.0,21.60,0.0,0.0,42.0,NaN,16.0,0.0,51507.0,24.0,19263.77,19263.77,10271.360000,10271.36,8736.23,1535.13,0.0,0.00,0.0000,858.05,0.0,NaN,1.0,NaN,NaN,0.0,0.0,221110.0,0.0,2.0,0.0,0.0,27.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,68495092.0,73384866.0,8650.0,8650.0,8650.0,19.89,320.99,55000.0,25.49,0.0,4.0,NaN,30.0,18.0,1.0,9568.0,19.0,0.00,0.00,9190.490000,9190.49,8650.00,540.49,0.0,0.00,0.0000,8251.42,0.0,NaN,1.0,NaN,NaN,0.0,0.0,18926.0,6.0,1.0,0.0,1.0,19.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,68506798.0,73396623.0,23000.0,23000.0,23000.0,8.49,471.77,64000.0,18.28,0.0,0.0,29.0,NaN,21.0,0.0,24940.0,33.0,19143.69,19143.69,5634.120000,5634.12,3856.31,1777.81,0.0,0.00,0.0000,471.77,0.0,NaN,1.0,NaN,NaN,0.0,0.0,63886.0,0.0,7.0,0.0,0.0,42.0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,68566886.0,73456723.0,29900.0,29900.0,29900.0,12.88,678.49,65000.0,21.77,0.0,0.0,NaN,NaN,15.0,0.0,31682.0,31.0,25346.66,25346.66,8120.480000,8120.48,4553.34,3567.14,0.0,0.00,0.0000,678.49,0.0,NaN,1.0,NaN,NaN,0.0,0.0,220251.0,2.0,1.0,1.0,2.0,9.0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
7,68577849.0,73467703.0,18000.0,18000.0,18000.0,11.99,40

It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [10]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.97964902, 0.98076512, 0.98162   , 0.98178623, 0.96179055,
       0.9760152 , 0.96050725, 0.98064547, 0.97993208, 0.98050159])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?


First check all of the possible loan statuses to get an idea of what loan status indicates and how often each status occurs.

In [11]:
y2015['loan_status'].value_counts()

Current               287414
Fully Paid             87989
Charged Off            29178
Late (31-120 days)      9510
In Grace Period         4320
Late (16-30 days)       1888
Default                  796
Name: loan_status, dtype: int64

For two loan statuses (Fully Paid, Charged Off), no further payments are expected while four of the loan statuses (Current, In Grace Period, Late (16-30 days) and Late (31-120) days) are related to last payment . It is unclear how to differentiate default loans but these only constitute a small portion of the data set. This suggests that payment amount and outstanding principal may not be needed as these amounts do not effect the actual loan statuses, although outstanding principal of 0 does indicate a loan that has been fully paid or charged off. Payment amount is not relevant

In [12]:
data = pd.DataFrame()
data['loan_status'] = y2015['loan_status']

For the two statuses with no further payment, the 'next_pymnt_d' is not listed (i.e. NaN).

In [13]:
data['no_next_payment'] = y2015['next_pymnt_d'].isna()==1
data['no_next_payment'] = data['no_next_payment'].astype(int)

The majority of charged off loans have non-zero amounts in 'recoveries'. For simplification, turn this into a bool and ignore magnitude.

In [14]:
data['recoveries'] = y2015['recoveries']
data['recoveries'] = data['recoveries'] > 0
data['recoveries'] = data['recoveries'].astype(int)

For the statuses based on last payment date, days since last payment needs to be calculated. Since payment dates are only listed by month and not exact date, these are only approximate but assume all dates are for the first day of the month. If the actual date is later, this will cause misclassification near each threshold, particularly those under a month. For example, some loans in the grace period may be classified as current even though they are a few days late. However, there are far more current loans than loans in the grace period so this should have a relatively minor effect.

In [15]:
last_payment_date_str = y2015['last_pymnt_d']
last_payment_date = pd.to_datetime(last_payment_date_str)

In [16]:
from datetime import datetime
data['last_payment_date'] = last_payment_date.astype('int64')//1e9
data['overdue_days'] = datetime(2017,1,1) - last_payment_date
data['overdue_days'] = data['overdue_days'].dt.total_seconds()/24/60/60

Run the model:

In [17]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

rfc = ensemble.RandomForestClassifier()
X = data[['no_next_payment','recoveries','overdue_days']]
Y = data['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.90707891, 0.90983354, 0.9101185 , 0.91539028, 0.91802422,
       0.91935407, 0.9205633 , 0.92182194, 0.92319567, 0.92564005])

The cross validation scores are all over 90% with relatively little variation using only three variables: next_pymnt_d, last_pymnt_d and recoveries.

Test using only two variables:

In [18]:
rfc = ensemble.RandomForestClassifier()
X = data[['no_next_payment','overdue_days']]
Y = data['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.89147729, 0.89147729, 0.89147729, 0.89147729, 0.89147471,
       0.89147471, 0.89149588, 0.89149588, 0.89153822, 0.89153565])

In [19]:
rfc = ensemble.RandomForestClassifier()
X = data[['no_next_payment','recoveries']]
Y = data['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.90707891, 0.90983354, 0.9101185 , 0.91539028, 0.91802422,
       0.91935407, 0.9205633 , 0.92182194, 0.92319567, 0.92564005])

In [20]:
rfc = ensemble.RandomForestClassifier()
X = data[['recoveries','overdue_days']]
Y = data['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.69813113, 0.70088575, 0.70117072, 0.7064425 , 0.70907148,
       0.71040133, 0.7116056 , 0.71286423, 0.71422804, 0.71669121])

Both models using no_next_payment and one of the other variables have accuracy around 90% while the last model only has an accuracy around 70%.

Using a single variable:

In [21]:
rfc = ensemble.RandomForestClassifier()
X = data['no_next_payment']
Y = data['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.89147729, 0.89147729, 0.89147729, 0.89147729, 0.89147471,
       0.89147471, 0.89149588, 0.89149588, 0.89153822, 0.89153565])

In [22]:
rfc = ensemble.RandomForestClassifier()
X = data['overdue_days']
Y = data['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.86362233, 0.86547458, 0.86865665, 0.87048515, 0.87202565,
       0.87387794, 0.87475362, 0.8765822 , 0.87807253, 0.87683466])

In [23]:
rfc = ensemble.RandomForestClassifier()
X = data['recoveries']
Y = data['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.69813113, 0.70088575, 0.70117072, 0.7064425 , 0.70907148,
       0.71040133, 0.7116056 , 0.71286423, 0.71422804, 0.71669121])

Using only next_pymnt_dt or last_pymnt_dt yields accuracy between 86-89% while recoveries yields an accuracy around 70%.

Applying this to the original data yields the same result:

In [24]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

rfc = ensemble.RandomForestClassifier()
X = y2015[['next_pymnt_d','recoveries']]
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.90707891, 0.90983354, 0.9101185 , 0.91539028, 0.91802422,
       0.91935407, 0.9205633 , 0.92182194, 0.92319567, 0.9256163 ])

In [25]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

rfc = ensemble.RandomForestClassifier()
X = y2015['next_pymnt_d']
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.89147729, 0.89147729, 0.89147729, 0.89147729, 0.89147471,
       0.89147471, 0.89149588, 0.89149588, 0.89153822, 0.8915119 ])

Looking back at the original loan status frequencies, this is actually not surprising because 89% of loans fall in two categories (current and fully paid off) and these can be differentiated by next payment date. Just assuming all loans with a next payment date are current (including those that are late) and loans without a next payment date are fully paid (including those that have actually been charged off) would give an accuracy of 89%. This is indicative of the unbalanced nature of the data set. However, adding a couple of more features can increase accuracy. Recoveries is important in this data set because the number of charged off loans is a significant portion (25%) of the loans with no next payment date while overdue loans are a much smaller percentage (5%) of loans with a next payment date.

In [26]:
y2015['loan_status'].value_counts()

Current               287414
Fully Paid             87989
Charged Off            29178
Late (31-120 days)      9510
In Grace Period         4320
Late (16-30 days)       1888
Default                  796
Name: loan_status, dtype: int64

In [27]:
sum(y2015['loan_status'] == 'Current') / len(y2015) + sum(y2015['loan_status'] == 'Fully Paid') / len(y2015)

0.8914924185753809